In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timezone
from azure.cosmos import CosmosClient
from utilities.constants import (
    azure_cosmosdb_endpoint_prod,
    azure_cosmosdb_account_key_prod,
    azure_cosmosdb_database_prod,
    azure_cosmosdb_conversations_container_prod,
    logging
)

# Setup
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)




In [ ]:
# Initialize Cosmos DB client
cosmos_client = CosmosClient(azure_cosmosdb_endpoint_prod, azure_cosmosdb_account_key_prod)
database = cosmos_client.get_database_client(azure_cosmosdb_database_prod)
container = database.get_container_client(azure_cosmosdb_conversations_container_prod)

In [ ]:

# Define date range (May 1 to May 15, 2025)
start_date = datetime(2025, 5, 1, tzinfo=timezone.utc)
end_date = datetime(2025, 5, 15, 23, 59, 59, tzinfo=timezone.utc)


In [ ]:

# Cosmos DB query
query_detailed = """
    SELECT  *
    FROM c
    WHERE c.createdAt >= '2025-05-01' AND c.createdAt <= '2025-05-15T23:59:59Z'
    ORDER BY c.updatedAt DESC
"""

In [ ]:

chat_logs = list(container.query_items(query=query_detailed, enable_cross_partition_query=True))
df_data = pd.DataFrame(chat_logs)

In [ ]:
# Helper functions
def feedback(row):
    if row['thumpsUp'] == 0.0 and row['thumpsDown'] == 0.0:
        return "no-feedback"
    elif row['thumpsUp'] == 1.0 and row['thumpsDown'] == 0.0:
        return "thumpsUp"
    elif row['thumpsUp'] == 0.0 and row['thumpsDown'] == 1.0:
        return "thumpsDown"
    else:
        return "no-feedback"

def month_week(dates):
    firstday_month = dates - pd.to_timedelta(dates.dt.day - 1, unit='d')
    return (dates.dt.day - 1 + firstday_month.dt.weekday) // 7 + 1


In [ ]:

# Date parsing
df_data['createdAt'] = pd.to_datetime(df_data['createdAt'], format='mixed')
df_data = df_data[(df_data['createdAt'] >= start_date) & (df_data['createdAt'] <= end_date)]

# Extract date parts
df_data['created_date'] = df_data['createdAt'].dt.date
df_data['month'] = df_data['createdAt'].dt.month
df_data['week_of_month'] = month_week(df_data['createdAt'])
df_data['month-week'] = df_data['month'].astype(str) + '-' + df_data['week_of_month'].astype(str)

# Feedback label
df_data['feedback'] = df_data.apply(feedback, axis=1)
df_data[['input_token_count','output_tokens','total_tokens','responseTime','thumpsUp','thumpsDown']] = \
    df_data[['input_token_count','output_tokens','total_tokens','responseTime','thumpsUp','thumpsDown']].fillna(0.0)

# Flatten
df_data_flat = df_data[['userId', 'id', 'type','created_date','month','week_of_month','month-week','user_query', 'output_response',
                        'chat_id', 'input_token_count','output_tokens','total_tokens','responseTime', 'thumpsUp','thumpsDown','feedback']]

In [ ]:

# ──────────────────────────────────────────────────────────────
# METRIC BLOCKS
# ──────────────────────────────────────────────────────────────

# 1. Feedback Counts per Week
feedback_categories = ['no-feedback', 'thumpsDown', 'thumpsUp']
feedback_summary = (
    df_data_flat.groupby('month-week')['feedback']
    .value_counts()
    .unstack()
    .reindex(columns=feedback_categories, fill_value=0)
)
print("\n🔹 Feedback Summary")
print(feedback_summary)

In [ ]:
# 2. Response Time Distribution
bins = [1, 10, 30, 60, 10000]
response_time_dist = df_data_flat.groupby(['month-week', pd.cut(df_data_flat.responseTime, bins)]).size().unstack(fill_value=0)
print("\n🔹 Response Time Distribution (sec bins)")
print(response_time_dist)


In [ ]:
# 3. Token Usage Summary
token_summary = df_data_flat.groupby('month-week').agg(
    inputTokens=('input_token_count', 'sum'),
    outputTokens=('output_tokens', 'sum'),
    totalTokens=('total_tokens', 'sum')
).reset_index()
print("\n Token Summary")
print(token_summary)


In [ ]:

# 4. Average Model Response Time per Query
avg_response_summary = df_data_flat.groupby('month-week').agg(
    avg_model_response_time=('responseTime', 'mean'),
    query_count=('user_query', 'size')
).reset_index()
print("\nAvg Model Response Time per Query")
print(avg_response_summary)


In [ ]:
# 5. Active Users Weekly (3+ queries)
df_weekly_users = df_data_flat.groupby(['month','week_of_month','userId']).size().reset_index(name='query_count')
active_users_summary = df_weekly_users[df_weekly_users['query_count'] >= 3].groupby(['month','week_of_month']).agg(
    active_users=('userId', 'size'),
    active_users_list=('userId', lambda x: list(x.unique()))
).reset_index()
print("\n Active Users (>=3 queries)")
print(active_users_summary)

In [ ]:


# ──────────────────────────────────────────────────────────────
#  Visuals (Feedback + Response Time)
# ──────────────────────────────────────────────────────────────

# Feedback Plot
feedback_summary.plot(kind='bar', stacked=True)
plt.title('Feedback by Month-Week')
plt.xlabel('Month-Week')
plt.ylabel('Count')
plt.tight_layout()
plt.show()


In [ ]:

# Response Time Plot
ax = response_time_dist.plot(kind='bar', stacked=False)
for p in ax.patches:
    height = p.get_height()
    if height > 0:
        ax.annotate(str(height), (p.get_x() + p.get_width() / 2., height), ha='center', va='bottom')
plt.title('Response Time Buckets by Week')
plt.xlabel('Month-Week')
plt.ylabel('Query Count')
plt.legend(['1-10s', '10-30s', '30-60s', '60+ sec'])
plt.tight_layout()
plt.show()
